![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
from AlgorithmImports import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go
from datetime import datetime
from itertools import groupby


# --- 1. 初始化和历史数据请求 ---
qb = QuantBook()

symbol = qb.add_future(Futures.Indices.NASDAQ_100_E_MINI, Resolution.MINUTE,extended_market_hours=True).symbol
qb.set_time_zone("America/New_York")
start = date(2016, 1, 1)
end = date(2016, 12, 31)

minute_bars = qb.history[TradeBar](symbol, start, end, Resolution.MINUTE,
                    extended_market_hours=True,
                    data_mapping_mode=DataMappingMode.OPEN_INTEREST_ANNUAL, 
                    data_normalization_mode=DataNormalizationMode.RAW, 
                    fill_forward=True
                    )
# for bar  in tradebar_list:
#     print(bar.time,bar.end_time , bar)
for current_date, bars_in_day in groupby(minute_bars, key=lambda x: x.Time.date()):
    first_minute_bar = next(bars_in_day, None)

    if not first_minute_bar:
        continue
    
    year = current_date.year
    trade_date_int = int(current_date.strftime("%Y%m%d"))
    
    daily_open = first_minute_bar.Open
    print(first_minute_bar.time,first_minute_bar.end_time , first_minute_bar)